In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/jorge/thesis')


from text3d2video.artifacts.anim_artifact import AnimationArtifact
from torch import Tensor
import torchvision.transforms.functional as TF
import torch

In [49]:
from PIL import Image

from text3d2video.utilities.ipython_utils import display_vid
from text3d2video.utilities.video_util import pil_frames_to_clip

noises = torch.randn(5, 4, 64, 64)

def noise_imgs(noises: Tensor, res: int=512):

    images = []
    for noise in noises:
        noise_pil = TF.to_pil_image(noise[0:3, :, :].cpu())
        noise_pil = noise_pil.resize((res, res), Image.NEAREST)
        images.append(noise_pil)
    return images

display_vid(pil_frames_to_clip(noise_imgs(noises)))

In [50]:
from text3d2video.noise_initialization import RandomNoiseInitializer
from torch import Generator

device = 'cuda'

gen = Generator(device=device)
gen.manual_seed(0)

random_noise = RandomNoiseInitializer()

noise = random_noise.initial_noise(4, generator=gen, device=device)

display_vid(pil_frames_to_clip(noise_imgs(noise)))

In [4]:
from text3d2video.noise_initialization import FixedNoiseInitializer, RandomNoiseInitializer
from torch import Generator

device = 'cuda'

gen = Generator(device=device)
gen.manual_seed(0)

random_noise = FixedNoiseInitializer()

noise = random_noise.initial_noise(4, generator=gen, device=device)

display_vid(pil_frames_to_clip(noise_imgs(noise)))

In [7]:
from text3d2video.noise_initialization import (
    UVNoiseInitializer,
)
from torch import Generator

animation = AnimationArtifact.from_wandb_artifact_tag("mixamo-human_orth_pan:latest")

n_images = 20

frame_indices = animation.frame_indices(n_images)
cameras, meshes = animation.load_frames(frame_indices)
verts_uvs, faces_uvs = animation.uv_data()

# depth_maps = render_depth_map(meshes, cameras)

device = "cuda"

gen = Generator(device=device)
gen.manual_seed(0)

random_noise = UVNoiseInitializer(noise_texture_res=50)

noise = random_noise.initial_noise(meshes, cameras, verts_uvs, faces_uvs, generator=gen)

display_vid(pil_frames_to_clip(noise_imgs(noise)))

In [51]:
noise_init = UVNoiseInitializer()

gen_bg = torch.Generator(device="cuda").manual_seed(1)
gen_tex = torch.Generator(device="cuda").manual_seed(1)

noise_init.sample_background(generator=gen_bg)
noise_init.sample_noise_texture(generator=gen_tex)

noise = noise_init.initial_noise(meshes, cameras, verts_uvs, faces_uvs)

display_vid(pil_frames_to_clip(noise_imgs(noise)))

In [ ]:
uv_noise = noise_init.uv_noise
bg_noise = noise_init.bg_noise

reconstructed = UVNoiseInitializer.init_from_textures(uv_noise, bg_noise)

noise = reconstructed.initial_noise(meshes, cameras, verts_uvs, faces_uvs)
display_vid(pil_frames_to_clip(noise_imgs(noise)))

: 